In [1]:
import numpy as np
import math
import multiprocessing as mp

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from collections import namedtuple

from PIL import Image
import os
import os.path
import errno
import codecs
import copy

In [2]:
torch.manual_seed(0)
np.random.seed(0)
print("torch.cuda.device_count()", torch.cuda.device_count())
print("torch.cuda.current_device()", torch.cuda.current_device())
torch.cuda.set_device(2)
print("torch.cuda.current_device()", torch.cuda.current_device())

torch.cuda.device_count() 4
torch.cuda.current_device() 0
torch.cuda.current_device() 2


In [34]:
def compute_ranks(x):
  """
  Returns ranks in [0, len(x))
  Note: This is different from scipy.stats.rankdata, which returns ranks in [1, len(x)].
  (https://github.com/openai/evolution-strategies-starter/blob/master/es_distributed/es.py)
  """
  assert x.ndim == 1
  ranks = np.empty(len(x), dtype=int)
  ranks[x.argsort()] = np.arange(len(x))
  return ranks

def compute_centered_ranks(x):
  """
  https://github.com/openai/evolution-strategies-starter/blob/master/es_distributed/es.py
  """
  y = compute_ranks(x.ravel()).reshape(x.shape).astype(np.float32)
  y /= (x.size - 1)
  y -= .5
  return y

def compute_weight_decay(weight_decay, model_param_list):
  model_param_grid = np.array(model_param_list)
  return - weight_decay * np.mean(model_param_grid * model_param_grid, axis=1)

class CMAES:
  '''CMA-ES wrapper.'''
  def __init__(self, num_params,      # number of model parameters
               sigma_init=0.10,       # initial standard deviation
               popsize=255):          # population size

    self.num_params = num_params
    self.sigma_init = sigma_init
    self.popsize = popsize

    self.solutions = None

    import cma
    self.es = cma.CMAEvolutionStrategy( self.num_params * [0],
                                        self.sigma_init,
                                        {'popsize': self.popsize})

  def rms_stdev(self):
    sigma = self.es.result()[6]
    return np.mean(np.sqrt(sigma*sigma))

  def ask(self):
    '''returns a list of parameters'''
    self.solutions = np.array(self.es.ask())
    return self.solutions

  def tell(self, reward_table_result):
    reward_table = reward_table_result
    self.es.tell(self.solutions, (-reward_table).tolist()) # convert minimizer to maximizer.

  def done(self):
    return self.es.stop()

  def current_param(self):
    return self.es.result()[5] # mean solution, presumably better with noise
  
  def best_param(self):
    return self.es.result()[0] # best evaluated solution

  def result(self): # return best params so far, along with historically best reward, curr reward, sigma
    r = self.es.result()
    return (r[0], -r[1], -r[1], r[6])

class SimpleES:
  '''Simple Evolution Strategies.'''
  def __init__(self, num_params,      # number of model parameters
               sigma_init=0.10,       # initial standard deviation
               sigma_alpha=0.20,      # learning rate for standard deviation
               sigma_decay=0.999,     # anneal standard deviation
               sigma_limit=0.01,      # stop annealing if less than this
               popsize=255,           # population size
               elite_ratio=0.1,       # percentage of the elites
               done_threshold=1e-6,   # threshold when we say we are done
               average_baseline=True, # set baseline to average of batch
               forget_best=True):     # only use the best from latest generation

    self.num_params = num_params
    self.sigma_init = sigma_init
    self.sigma_alpha = sigma_alpha
    self.sigma_decay = sigma_decay
    self.sigma_limit = sigma_limit
    self.popsize = popsize
    self.average_baseline = average_baseline
    if self.average_baseline:
      assert (self.popsize & 2), "Population size must be even"
      self.batch_size = int(self.popsize / 2)
    else:
      assert (self.popsize & 1), "Population size must be odd"
      self.batch_size = int((self.popsize - 1) / 2)
    self.elite_ratio = elite_ratio
    self.elite_popsize = int(self.popsize * self.elite_ratio)
    self.forget_best = forget_best
    self.batch_reward = np.zeros(self.batch_size * 2)
    self.mu = np.zeros(self.num_params)
    self.sigma = np.ones(self.num_params) * self.sigma_init
    self.curr_best_mu = np.zeros(self.num_params)
    self.best_mu = np.zeros(self.num_params)
    self.best_reward = 0
    self.first_interation = True
    self.done_threshold = done_threshold

  def rms_stdev(self):
    sigma = self.sigma
    return np.mean(np.sqrt(sigma*sigma))

  def ask(self):
    '''returns a list of parameters'''
    # antithetic sampling
    self.epsilon = np.random.randn(self.batch_size, self.num_params) * self.sigma.reshape(1, self.num_params)
    self.epsilon_full = np.concatenate([self.epsilon, - self.epsilon])
    if self.average_baseline:
      epsilon = self.epsilon_full
    else:
      # first population is mu, then positive epsilon, then negative epsilon
      epsilon = np.concatenate([np.zeros((1, self.num_params)), self.epsilon_full])
    solutions = self.mu.reshape(1, self.num_params) + epsilon
    return solutions

  def tell(self, reward_table_result):
    # input must be a numpy float array
    assert(len(reward_table_result) == self.popsize), "Inconsistent reward_table size reported."

    reward_table = reward_table_result

    reward_offset = 1
    if self.average_baseline:
      b = np.mean(reward_table)
      reward_offset = 0
    else:
      b = reward_table[0] # baseline
      
    reward = reward_table[reward_offset:]
    idx = np.argsort(reward)[::-1][0:self.elite_popsize]

    best_reward = reward[idx[0]]
    if (best_reward > b or self.average_baseline):
      best_mu = self.mu + self.epsilon_full[idx[0]]
      best_reward = reward[idx[0]]
    else:
      best_mu = self.mu
      best_reward = b

    self.curr_best_reward = best_reward
    self.curr_best_mu = best_mu

    if self.first_interation:
      self.first_interation = False
      self.best_reward = self.curr_best_reward
      self.best_mu = best_mu
    else:
      if self.forget_best or (self.curr_best_reward > self.best_reward):
        self.best_mu = best_mu
        self.best_reward = self.curr_best_reward

    # adaptive sigma
    # normalization
    stdev_reward = reward.std()
    epsilon = self.epsilon
    sigma = self.sigma
    S = ((epsilon * epsilon - (sigma * sigma).reshape(1, self.num_params)) / sigma.reshape(1, self.num_params))
    reward_avg = (reward[:self.batch_size] + reward[self.batch_size:]) / 2.0
    rS = reward_avg - b
    delta_sigma = (np.dot(rS, S)) / (2 * self.batch_size * stdev_reward)

    # move mean to the average of the best idx means
    self.mu += self.epsilon_full[idx].mean(axis=0)

    # adjust sigma according to the adaptive sigma calculation
    change_sigma = self.sigma_alpha * delta_sigma
    change_sigma = np.minimum(change_sigma, self.sigma)
    change_sigma = np.maximum(change_sigma, - 0.5 * self.sigma)
    self.sigma += change_sigma
    self.sigma[self.sigma > self.sigma_limit] *= self.sigma_decay

  def done(self):
    return (self.rms_stdev() < self.done_threshold)

  def current_param(self):
    return self.curr_best_mu
  
  def best_param(self):
    return self.best_mu

  def result(self): # return best params so far, along with historically best reward, curr reward, sigma
    return (self.best_mu, self.best_reward, self.curr_best_reward, self.sigma)

class SimpleGA:
  '''Simple Genetic Algorithm.'''
  def __init__(self, num_params,      # number of model parameters
               sigma_init=0.1,        # initial standard deviation
               sigma_decay=0.999,     # anneal standard deviation
               sigma_limit=0.01,      # stop annealing if less than this
               popsize=255,           # population size
               elite_ratio=0.1,       # percentage of the elites
               forget_best=False,     # forget the historical best elites
               done_threshold=1e-6):  # threshold when we say we are done

    self.num_params = num_params
    self.sigma_init = sigma_init
    self.sigma_decay = sigma_decay
    self.sigma_limit = sigma_limit
    self.popsize = popsize

    self.elite_ratio = elite_ratio
    self.elite_popsize = int(self.popsize * self.elite_ratio)

    self.sigma = self.sigma_init
    self.elite_params = np.zeros((self.elite_popsize, self.num_params))
    self.elite_rewards = np.zeros(self.elite_popsize)
    self.best_param = np.zeros(self.num_params)
    self.best_reward = 0
    self.first_iteration = True
    self.forget_best = forget_best
    self.done_threshold = done_threshold

  def rms_stdev(self):
    return self.sigma # same sigma for all parameters.

  def ask(self):
    '''returns a list of parameters'''
    # antithetic sampling
    self.epsilon = np.random.randn(self.popsize, self.num_params) * self.sigma
    solutions = []
    
    def mate(a, b):
      c = np.copy(a)
      idx = np.where(np.random.rand((c.size)) > 0.5)
      c[idx] = b[idx]
      return c
    
    elite_range = range(self.elite_popsize)
    for i in range(self.popsize):
      idx_a = np.random.choice(elite_range)
      idx_b = np.random.choice(elite_range)
      child_params = mate(self.elite_params[idx_a], self.elite_params[idx_b])
      solutions.append(child_params + self.epsilon[i])

    solutions = np.array(solutions)
    self.solutions = solutions

    return solutions

  def tell(self, reward_table_result):
    # input must be a numpy float array
    assert(len(reward_table_result) == self.popsize), "Inconsistent reward_table size reported."
    
    if (not self.forget_best or self.first_iteration):
      reward = reward_table_result
      solution = self.solutions
    else:
      reward = np.concatenate([reward_table_result, self.elite_rewards])
      solution = np.concatenate([self.solutions, self.elite_params])

    idx = np.argsort(reward)[::-1][0:self.elite_popsize]

    self.elite_rewards = reward[idx]
    self.elite_params = solution[idx]

    self.curr_best_reward = self.elite_rewards[0]
    
    if self.first_iteration or (self.curr_best_reward > self.best_reward):
      self.first_iteration = False
      self.best_reward = self.elite_rewards[0]
      self.best_param = np.copy(self.elite_params[0])

    if (self.sigma > self.sigma_limit):
      self.sigma *= self.sigma_decay

  def done(self):
    return (self.rms_stdev() < self.done_threshold)

  def current_param(self):
    return self.elite_params[0]

  def best_param(self):
    return self.best_param

  def result(self): # return best params so far, along with historically best reward, curr reward, sigma
    return (self.best_param, self.best_reward, self.curr_best_reward, self.sigma)

class OpenES:
  ''' Basic Version of OpenAI Evolution Strategies.'''
  def __init__(self, num_params,             # number of model parameters
               sigma_init=0.1,               # initial standard deviation
               sigma_decay=0.999,            # anneal standard deviation
               sigma_limit=0.01,             # stop annealing if less than this
               learning_rate=0.001,          # learning rate for standard deviation
               learning_rate_decay = 0.9999, # annealing the learning rate
               learning_rate_limit = 0.001,  # stop annealing learning rate
               popsize=255,                  # population size
               antithetic=False,             # whether to use antithetic sampling
               forget_best=True):           # forget historical best

    self.num_params = num_params
    self.sigma_decay = sigma_decay
    self.sigma = sigma_init
    self.sigma_limit = sigma_limit
    self.learning_rate = learning_rate
    self.learning_rate_decay = learning_rate_decay
    self.learning_rate_limit = learning_rate_limit
    self.popsize = popsize
    self.antithetic = antithetic
    if self.antithetic:
      assert (self.popsize & 2), "Population size must be even"
      self.half_popsize = int(self.popsize / 2)

    self.reward = np.zeros(self.popsize)
    self.mu = np.zeros(self.num_params)
    self.best_mu = np.zeros(self.num_params)
    self.best_reward = 0
    self.first_interation = True
    self.forget_best = forget_best

  def rms_stdev(self):
    sigma = self.sigma
    return np.mean(np.sqrt(sigma*sigma))

  def ask(self):
    '''returns a list of parameters'''
    # antithetic sampling
    if self.antithetic:
      self.epsilon_half = np.random.randn(self.half_popsize, self.num_params)
      self.epsilon = np.concatenate([self.epsilon_half, - self.epsilon_half])
    else:
      self.epsilon = np.random.randn(self.popsize, self.num_params)

    self.solutions = self.mu.reshape(1, self.num_params) + self.epsilon * self.sigma

    return self.solutions

  def tell(self, reward):
    # input must be a numpy float array
    assert(len(reward) == self.popsize), "Inconsistent reward_table size reported."

    idx = np.argsort(reward)[::-1]

    best_reward = reward[idx[0]]
    best_mu = self.solutions[idx[0]]

    self.curr_best_reward = best_reward
    self.curr_best_mu = best_mu

    if self.first_interation:
      self.first_interation = False
      self.best_reward = self.curr_best_reward
      self.best_mu = best_mu
    else:
      if self.forget_best or (self.curr_best_reward > self.best_reward):
        self.best_mu = best_mu
        self.best_reward = self.curr_best_reward

    # main bit:
    # standardize the rewards to have a gaussian distribution
    normalized_reward = (reward - np.mean(reward)) / np.std(reward)
    self.mu += self.learning_rate/(self.popsize*self.sigma)*np.dot(self.epsilon.T, normalized_reward)

    # adjust sigma according to the adaptive sigma calculation
    if (self.sigma > self.sigma_limit):
      self.sigma *= self.sigma_decay

    if (self.learning_rate > self.learning_rate_limit):
      self.learning_rate *= self.learning_rate_decay

  def done(self):
    return False

  def current_param(self):
    return self.curr_best_mu

  def best_param(self):
    return self.best_mu

  def result(self): # return best params so far, along with historically best reward, curr reward, sigma
    return (self.best_mu, self.best_reward, self.curr_best_reward, self.sigma)

In [35]:
Args = namedtuple('Args', ['batch_size', 'test_batch_size', 'epochs', 'lr', 'cuda', 'seed', 'log_interval'])

In [36]:
args = Args(batch_size=100, test_batch_size=1000, epochs=30, lr=0.001, cuda=True, seed=0, log_interval=10)

In [37]:
torch.manual_seed(args.seed)
if args.cuda:
  torch.cuda.manual_seed(args.seed)

In [38]:
kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}

train_loader = torch.utils.data.DataLoader(
  datasets.MNIST('MNIST_data', train=True, download=True, transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])),
  batch_size=args.batch_size, shuffle=True, **kwargs)

valid_loader = train_loader

test_loader = torch.utils.data.DataLoader(
  datasets.MNIST('MNIST_data', train=False, transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])),
  batch_size=args.batch_size, shuffle=True, **kwargs)

In [39]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.num_filter1 = 8
    self.num_filter2 = 16
    self.num_padding = 2
    # input is 28x28
    # padding=2 for same padding
    self.conv1 = nn.Conv2d(1, self.num_filter1, 5, padding=self.num_padding)
    # feature map size is 14*14 by pooling
    # padding=2 for same padding
    self.conv2 = nn.Conv2d(self.num_filter1, self.num_filter2, 5, padding=self.num_padding)
    # feature map size is 7*7 by pooling
    self.fc = nn.Linear(self.num_filter2*7*7, 10)

  def forward(self, x):
    x = F.max_pool2d(F.relu(self.conv1(x)), 2)
    x = F.max_pool2d(F.relu(self.conv2(x)), 2)
    x = x.view(-1, self.num_filter2*7*7)   # reshape Variable
    x = self.fc(x)
    return F.log_softmax(x)

In [40]:
NPOPULATION = 101
weight_decay_coef = 0.1

In [41]:
'''
models = []
for i in range(NPOPULATION):
  model = Net()
  if args.cuda:
    model.cuda()
  model.eval()
  models.append(model)
'''

model = Net()
if args.cuda:
  model.cuda()

orig_model = copy.deepcopy(model)

In [42]:
# get init params
orig_params = []
model_shapes = []
for param in orig_model.parameters():
  p = param.data.cpu().numpy()
  model_shapes.append(p.shape)
  orig_params.append(p.flatten())
orig_params_flat = np.concatenate(orig_params)
NPARAMS = len(orig_params_flat)
print(NPARAMS)

11274


In [43]:
def update_model(flat_param, model, model_shapes):
  idx = 0
  i = 0
  for param in model.parameters():
    delta = np.product(model_shapes[i])
    block = flat_param[idx:idx+delta]
    block = np.reshape(block, model_shapes[i])
    i += 1
    idx += delta
    block_data = torch.from_numpy(block).float()
    if args.cuda:
      block_data = block_data.cuda()
    param.data = block_data

In [44]:
def evaluate(model, test_loader, print_mode=True, return_loss=False):
  model.eval()
  test_loss = 0
  correct = 0
  for data, target in test_loader:
    if args.cuda:
      data, target = data.cuda(), target.cuda()
    data, target = Variable(data, volatile=True), Variable(target)
    output = model(data)
    test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
    pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
    correct += pred.eq(target.data.view_as(pred)).cpu().sum()

  test_loss /= len(test_loader.dataset)
  acc = correct / len(test_loader.dataset)
  
  if print_mode:
    print('\nAverage loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
      test_loss, correct, len(test_loader.dataset),
      100. * acc))
  
  if return_loss:
    return test_loss
  return acc

In [48]:
"""
es = SimpleES(NPARAMS,
              popsize=NPOPULATION,
              sigma_init=0.01,
              sigma_decay=0.999,
              sigma_alpha=0.2,
              sigma_limit=0.001,
              elite_ratio=0.1,
              average_baseline=False,
              forget_best=True
             )
"""
es = SimpleGA(NPARAMS,
              popsize=NPOPULATION,
              forget_best=False,
              sigma_init=0.01,
              sigma_decay=0.9999,
              sigma_limit=0.01
             )

In [49]:
def worker(procnum, model, solution, data, target, send_end):
  update_model(solution, model, model_shapes)
  output = model(data)
  loss = F.nll_loss(output, target)
  reward = - loss.data[0]
  send_end.send(reward)

def batch_simulation(model_list, solutions, data, target, process_count):
  jobs = []
  pipe_list = []

  for i in range(process_count):
    recv_end, send_end = mp.Pipe(False)
    p = mp.Process(target=worker, args=(i, model_list[i], solutions[i], data, target, send_end))
    jobs.append(p)
    pipe_list.append(recv_end)

  for p in jobs:
    p.start()

  for p in jobs:
    p.join()

  result_list = [x.recv() for x in pipe_list]
  return np.array(result_list)


def batch_simulation_sequential(model_list, solutions, data, target, process_count):
  result_list = []
  for i in range(process_count):
    update_model(solutions[i], model_list[i], model_shapes)
    output = model_list[i](data)
    loss = F.nll_loss(output, target)
    reward = - loss.data[0]
    result_list.append(reward)
  return np.array(result_list)


In [50]:
#'''
best_valid_acc = 0
training_log = []
for epoch in range(1, 10*args.epochs + 1):

  # train loop
  model.eval()
  for batch_idx, (data, target) in enumerate(train_loader):
    if args.cuda:
      data, target = data.cuda(), target.cuda()
    data, target = Variable(data), Variable(target)
    
    solutions = es.ask()
    reward = np.zeros(es.popsize)
    
    for i in range(es.popsize):
      update_model(solutions[i], model, model_shapes)
      output = model(data)
      loss = F.nll_loss(output, target)
      reward[i] = - loss.data[0]

    best_raw_reward = reward.max()
    reward = compute_centered_ranks(reward)
    l2_decay = compute_weight_decay(weight_decay_coef, solutions)
    reward += l2_decay

    es.tell(reward)

    result = es.result()
    
    if (batch_idx % 50 == 0):
      print(epoch, batch_idx, best_raw_reward, result[0].mean(), result[3])

  curr_solution = es.current_param()
  update_model(curr_solution, model, model_shapes)

  valid_acc = evaluate(model, valid_loader, print_mode=False)
  training_log.append([epoch, valid_acc])
  print('valid_acc', valid_acc * 100.)
  if valid_acc >= best_valid_acc:
    best_valid_acc = valid_acc
    best_model = copy.deepcopy(model)
    print('best valid_acc', best_valid_acc * 100.)
#'''

1 0 -2.30060434341 -0.000112202587902 0.01
1 50 -1.20309865475 -0.000112202587902 0.01
1 100 -1.10738801956 -0.000112202587902 0.01
1 150 -0.919825077057 -0.000112202587902 0.01
1 200 -0.961853086948 -0.000112202587902 0.01
1 250 -0.976065456867 -0.000112202587902 0.01
1 300 -0.945845663548 -0.000112202587902 0.01
1 350 -1.14225947857 -0.000112202587902 0.01
1 400 -0.837524473667 -0.000112202587902 0.01
1 450 -0.634921133518 -0.000112202587902 0.01
1 500 -0.948218405247 -0.000112202587902 0.01
1 550 -0.961603403091 -0.000112202587902 0.01
valid_acc 80.205
best valid_acc 80.205
2 0 -0.757367372513 -0.000112202587902 0.01
2 50 -0.732529580593 -0.000112202587902 0.01
2 100 -1.03761339188 -0.000112202587902 0.01
2 150 -1.2254191637 -0.000112202587902 0.01
2 200 -0.758734345436 -0.000112202587902 0.01
2 250 -0.64583837986 -0.000112202587902 0.01
2 300 -1.1819319725 -0.000112202587902 0.01
2 350 -0.632461309433 -0.000112202587902 0.01
2 400 -0.862645924091 -0.000112202587902 0.01
2 450 -0.40

15 200 -0.645030379295 -0.000112202587902 0.01
15 250 -0.789413750172 -0.000112202587902 0.01
15 300 -0.825545430183 -0.000112202587902 0.01
15 350 -1.29622197151 -0.000112202587902 0.01
15 400 -0.878007531166 -0.000112202587902 0.01
15 450 -0.836985468864 -0.000112202587902 0.01
15 500 -0.520756959915 -0.000112202587902 0.01
15 550 -0.939542472363 -0.000112202587902 0.01
valid_acc 73.2
16 0 -1.11171543598 -0.000112202587902 0.01
16 50 -1.19556856155 -0.000112202587902 0.01
16 100 -0.803073704243 -0.000112202587902 0.01
16 150 -0.825310111046 -0.000112202587902 0.01
16 200 -0.85929608345 -0.000112202587902 0.01
16 250 -0.854680776596 -0.000112202587902 0.01
16 300 -1.10111641884 -0.000112202587902 0.01
16 350 -1.03547465801 -0.000112202587902 0.01
16 400 -1.06607413292 -0.000112202587902 0.01
16 450 -1.3802074194 -0.000112202587902 0.01
16 500 -1.16331291199 -0.000112202587902 0.01
16 550 -0.873036324978 -0.000112202587902 0.01
valid_acc 74.225
17 0 -0.676514804363 -0.000112202587902 0

29 300 -0.711340129375 -0.000112202587902 0.01
29 350 -0.941682636738 -0.000112202587902 0.01
29 400 -0.992071330547 -0.000112202587902 0.01
29 450 -1.1413217783 -0.000112202587902 0.01
29 500 -0.917519807816 -0.000112202587902 0.01
29 550 -1.01626849174 -0.000112202587902 0.01
valid_acc 66.39500000000001
30 0 -0.616011798382 -0.000112202587902 0.01
30 50 -0.956930339336 -0.000112202587902 0.01
30 100 -0.976789116859 -0.000112202587902 0.01
30 150 -0.94179046154 -0.000112202587902 0.01
30 200 -0.921755433083 -0.000112202587902 0.01
30 250 -0.907006382942 -0.000112202587902 0.01
30 300 -0.835935354233 -0.000112202587902 0.01
30 350 -0.976340353489 -0.000112202587902 0.01
30 400 -1.08026218414 -0.000112202587902 0.01
30 450 -1.57223188877 -0.000112202587902 0.01
30 500 -1.0037535429 -0.000112202587902 0.01
30 550 -0.827641427517 -0.000112202587902 0.01
valid_acc 71.87833333333333
31 0 -0.928116679192 -0.000112202587902 0.01
31 50 -0.773678004742 -0.000112202587902 0.01
31 100 -0.88463258

43 400 -1.21473741531 -0.000112202587902 0.01
43 450 -0.780671298504 -0.000112202587902 0.01
43 500 -0.763305485249 -0.000112202587902 0.01
43 550 -0.986675262451 -0.000112202587902 0.01
valid_acc 72.72333333333333
44 0 -0.560661792755 -0.000112202587902 0.01
44 50 -0.841722548008 -0.000112202587902 0.01
44 100 -0.659147262573 -0.000112202587902 0.01
44 150 -0.841374158859 -0.000112202587902 0.01
44 200 -1.29826307297 -0.000112202587902 0.01
44 250 -0.892942726612 -0.000112202587902 0.01
44 300 -0.700051546097 -0.000112202587902 0.01
44 350 -0.966121375561 -0.000112202587902 0.01
44 400 -0.716448187828 -0.000112202587902 0.01
44 450 -1.39912796021 -0.000112202587902 0.01
44 500 -0.946059465408 -0.000112202587902 0.01
44 550 -0.655905842781 -0.000112202587902 0.01
valid_acc 71.53500000000001
45 0 -0.772949278355 -0.000112202587902 0.01
45 50 -0.883020222187 -0.000112202587902 0.01
45 100 -1.06702172756 -0.000112202587902 0.01
45 150 -0.829590976238 -0.000112202587902 0.01
45 200 -0.8990

57 400 -0.482666105032 -0.000112202587902 0.01
57 450 -0.543205857277 -0.000112202587902 0.01
57 500 -0.713469982147 -0.000112202587902 0.01
57 550 -0.559324324131 -0.000112202587902 0.01
valid_acc 76.17833333333334
58 0 -0.470753580332 -0.000112202587902 0.01
58 50 -0.565346837044 -0.000112202587902 0.01
58 100 -0.499830126762 -0.000112202587902 0.01
58 150 -0.448119670153 -0.000112202587902 0.01
58 200 -0.893559336662 -0.000112202587902 0.01
58 250 -0.811080694199 -0.000112202587902 0.01
58 300 -0.904327690601 -0.000112202587902 0.01
58 350 -0.652942955494 -0.000112202587902 0.01
58 400 -0.729658067226 -0.000112202587902 0.01
58 450 -0.466366350651 -0.000112202587902 0.01
58 500 -0.338601529598 -0.000112202587902 0.01
58 550 -0.743045628071 -0.000112202587902 0.01
valid_acc 78.50333333333333
59 0 -0.585690975189 -0.000112202587902 0.01
59 50 -0.666297316551 -0.000112202587902 0.01
59 100 -0.636935472488 -0.000112202587902 0.01
59 150 -0.767390727997 -0.000112202587902 0.01
59 200 -0.

...

210 0 -0.685525655746 -0.000112202587902 0.01
210 50 -0.579519927502 -0.000112202587902 0.01
210 100 -0.603381276131 -0.000112202587902 0.01
210 150 -0.487536042929 -0.000112202587902 0.01
210 200 -0.699151694775 -0.000112202587902 0.01
210 250 -1.11324477196 -0.000112202587902 0.01
210 300 -0.303980916739 -0.000112202587902 0.01
210 350 -0.390177637339 -0.000112202587902 0.01
210 400 -1.00099658966 -0.000112202587902 0.01
210 450 -0.921086072922 -0.000112202587902 0.01
210 500 -0.559709727764 -0.000112202587902 0.01
210 550 -0.539414763451 -0.000112202587902 0.01
valid_acc 75.09333333333333
211 0 -0.78613692522 -0.000112202587902 0.01
211 50 -0.540627658367 -0.000112202587902 0.01
211 100 -0.97925812006 -0.000112202587902 0.01
211 150 -0.701588869095 -0.000112202587902 0.01
211 200 -0.420407801867 -0.000112202587902 0.01
211 250 -0.569962859154 -0.000112202587902 0.01
211 300 -0.759323060513 -0.000112202587902 0.01
211 350 -0.545364499092 -0.000112202587902 0.01
211 400 -0.75410902500

223 500 -0.87151992321 -0.000112202587902 0.01
223 550 -0.718733847141 -0.000112202587902 0.01
valid_acc 77.56666666666666
224 0 -0.619056940079 -0.000112202587902 0.01
224 50 -0.644972324371 -0.000112202587902 0.01
224 100 -0.754298865795 -0.000112202587902 0.01
224 150 -0.789782583714 -0.000112202587902 0.01
224 200 -0.410754054785 -0.000112202587902 0.01
224 250 -0.911396563053 -0.000112202587902 0.01
224 300 -1.26628601551 -0.000112202587902 0.01
224 350 -0.590975046158 -0.000112202587902 0.01
224 400 -0.636246263981 -0.000112202587902 0.01
224 450 -0.824734210968 -0.000112202587902 0.01
224 500 -0.592872679234 -0.000112202587902 0.01
224 550 -1.06981897354 -0.000112202587902 0.01
valid_acc 74.58500000000001
225 0 -0.853442013264 -0.000112202587902 0.01
225 50 -0.894848585129 -0.000112202587902 0.01
225 100 -0.962149679661 -0.000112202587902 0.01
225 150 -0.439244002104 -0.000112202587902 0.01
225 200 -0.730589091778 -0.000112202587902 0.01
225 250 -0.596615970135 -0.00011220258790

Process Process-565:
KeyboardInterrupt
Traceback (most recent call last):
  File "/home/hadavid/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()


KeyboardInterrupt: 

  File "/home/hadavid/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/hadavid/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 34, in _worker_loop
    r = index_queue.get()
  File "/home/hadavid/anaconda3/lib/python3.6/multiprocessing/queues.py", line 343, in get
    res = self._reader.recv_bytes()
  File "/home/hadavid/anaconda3/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/hadavid/anaconda3/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/hadavid/anaconda3/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)


In [ ]:
evaluate(best_model, valid_loader, print_mode=True)

In [ ]:
evaluate(best_model, test_loader, print_mode=True)

In [ ]:
evaluate(best_model, train_loader, print_mode=True)

In [ ]:
update_model(es.best_param(), model, model_shapes)

In [ ]:
evaluate(model, valid_loader, print_mode=True)

In [ ]:
evaluate(model, test_loader, print_mode=True)

In [ ]:
evaluate(model, train_loader, print_mode=True)

In [ ]:
update_model(es.current_param(), model, model_shapes)

In [ ]:
evaluate(model, valid_loader, print_mode=True)

In [ ]:
evaluate(model, test_loader, print_mode=True)

In [ ]:
evaluate(model, train_loader, print_mode=True)

In [ ]:
eval_acc = evaluate(best_model, test_loader)
print('final test acc', eval_acc * 100.)

In [ ]:
param_count = 0
for param in model.parameters():
  print(param.data.shape)
  param_count += np.product(param.data.shape)
print(param_count)

In [ ]:
orig_params = []
for param in orig_model.parameters():
  orig_params.append(param.data.cpu().numpy().flatten())

In [ ]:
orig_params_flat = np.concatenate(orig_params)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
_ = plt.hist(orig_params_flat, bins=200)
plt.show()

In [ ]:
final_params = []
for param in best_model.parameters():
  final_params.append(param.data.cpu().numpy().flatten())
final_params_flat = np.concatenate(final_params)

In [ ]:
_ = plt.hist(final_params_flat, bins=200)
plt.show()